In [ ]:
# Stats are broken down 3 different ways:
## Season totals
### 162-game pace
## Game-by-game: cumulative
## Game-by-game: 7-day moving average

# Add stabilization lines to season views

In [ ]:
pd.__version__

In [ ]:
from jesse import jesse
jesse.SetOptions()
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pybaseball as mlb
import mlbgame
from IPython.display import clear_output

import pybaseball as _pybaseball
import preprocess
from preprocess import service, model, dates
from process import batter_processing

In [ ]:
# this was build by preprocess
batter_raw = pd.read_csv('../data/csv/raw_batter_stats.csv')

In [ ]:
#batter_raw[batter_raw.Name == 'J.D. Martinez']

In [ ]:
batter_clean = batter_processing.RawBatter(batter_raw)
batter_clean.process_batter()
cln = batter_clean.get_processed_batter()
cln.sort_values(['NAME', 'DATE'], ascending=[False, True], inplace=True)

In [ ]:
gr = cln.groupby(['NAME', 'YEAR'], as_index=False)

In [ ]:
cln2 = gr.apply(lambda cln: pd.DataFrame({'NAME': cln['NAME']
             , 'YEAR' : cln['YEAR']
             , 'DATE': cln['DATE']
             , 'G' : cln['G']
             , 'TEAM': cln['TEAM']
             , 'H_CUMSUM': cln.H.cumsum()
             , 'H_7DAYROLL' : cln.H.rolling(7).sum()
             , 'BB_CUMSUM': cln.BB.cumsum()
             , 'BB_7DAYROLL' : cln.BB.rolling(7).sum()
             , 'HBP_CUMSUM': cln.HBP.cumsum()
             , 'HBP_7DAYROLL' : cln.HBP.rolling(7).sum()
             , 'SF_CUMSUM': cln.SF.cumsum()
             , 'SF_7DAYROLL' : cln.SF.rolling(7).sum()
             , 'AB_CUMSUM': cln.AB.cumsum()
             , 'AB_7DAYROLL' : cln.AB.rolling(7).sum()
             , 'PA_CUMSUM': cln.PA.cumsum()
             , 'PA_7DAYROLL' : cln.PA.rolling(7).sum()                              
             , 'TB_CUMSUM': cln.TB.cumsum()
             , 'TB_7DAYROLL' : cln.TB.rolling(7).sum()  
             , 'SB_CUMSUM': cln.SB.cumsum()
             , 'SB_7DAYROLL' : cln.SB.rolling(7).sum() 
             , 'HR_CUMSUM': cln.HR.cumsum()
             , 'HR_7DAYROLL' : cln.HR.rolling(7).sum() 
             , 'RBI_CUMSUM': cln.RBI.cumsum()
             , 'RBI_7DAYROLL' : cln.RBI.rolling(7).sum() 
             , 'R_CUMSUM': cln.R.cumsum()
             , 'R_7DAYROLL' : cln.R.rolling(7).sum()              
                                })).sort_values(['NAME', 'DATE'], ascending=False)

cln2['CUMSUM_OBP'] = (
    cln2.H_CUMSUM + cln2.BB_CUMSUM  + cln2.HBP_CUMSUM) / (
    cln2.AB_CUMSUM + cln2.SF_CUMSUM + cln2.BB_CUMSUM + cln2.HBP_CUMSUM)
cln2['CUMSUM_SLG'] = (cln2.TB_CUMSUM / cln2.AB_CUMSUM)
cln2['CUMSUM_OPS'] = cln2.CUMSUM_OBP + cln2.CUMSUM_SLG
cln2['CUMSUM_BAVG'] = cln2.H_CUMSUM / cln2.AB_CUMSUM

cln2['ROLLING_OBP'] = (
    cln2.H_7DAYROLL + cln2.BB_7DAYROLL + cln2.HBP_7DAYROLL) / (
    cln2.AB_7DAYROLL + cln2.SF_7DAYROLL + cln2.BB_7DAYROLL + cln2.HBP_7DAYROLL)
cln2['ROLLING_SLG'] = (cln2.TB_7DAYROLL / cln2.AB_7DAYROLL)
cln2['ROLLING_OPS'] = cln2.ROLLING_OBP + cln2.ROLLING_SLG
cln2['ROLLING_BAVG'] = cln2.H_7DAYROLL / cln2.AB_7DAYROLL

In [ ]:
test = cln[cln.NAME == 'J.D. Martinez']
test.tail(5)

In [ ]:
#test2 = cln2[cln2.NAME == 'J.D. Martinez']

In [ ]:
test3 = cln2[(cln2.NAME == 'J.D. Martinez') & (cln2.YEAR == 2018)]

In [ ]:
print(test3.shape)
test3.head()

## Begin graph tests

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import NumeralTickFormatter, BasicTickFormatter, ColumnDataSource

# Use this if you want to turn off the output_file functionality
# import bokeh.io
# bokeh.io.reset_output()
# bokeh.io.output_notebook()

### Batting average test graph

In [ ]:
# prepare some data
x = test3.DATE
day7 = test3.ROLLING_BAVG
seas = test3.CUMSUM_BAVG

# Consolidate multiple vectors into a source dict
source = ColumnDataSource(data=dict(
    x=test3.DATE,
    day7=test3.ROLLING_BAVG,
    seas=test3.CUMSUM_BAVG,
))

# If I want to output the plot to html or save as a file
# output to static HTML file
# output_file("lines.html")

# create a plot
p = figure(title="simple line example"
           , x_axis_label='DATE'
           , y_axis_label='AVG'
           , x_axis_type='datetime'
           , y_axis_type='linear'
           , y_minor_ticks = 5
           , plot_width = 900
           , plot_height = 600
          )
# format the y axis
p.yaxis.formatter=NumeralTickFormatter(format='0.000')

# add a line renderer with legend and line thickness
p.line(x, day7, legend="Batting Avg: 7-day", line_width=2, color="teal", alpha=.4)
p.line(x, seas, legend="Batting Avg: season", line_width=3, color="coral")

# # add a second line
# p.vline_stack(['day7', 'seas'], x='x', source=source, color=["teal", "coral"]
#               , line_width=[2,3], alpha=[0.4, 1])


# show the results
show(p)


### Test building a graph function

In [ ]:
# prepare some data
x = test3.DATE
day7 = test3.ROLLING_BAVG
seas = test3.CUMSUM_BAVG

In [ ]:
stat_ref = {'HR':{'day7':'HR_7DAYROLL', 'seas':'HR_CUMSUM'}
            , 'SB':{'day7': 'SB_7DAYROLL', 'seas':'SB_CUMSUM'}}

In [ ]:
mx = cln2.HR_7DAYROLL.max()

In [ ]:
mx

In [ ]:
stat = 'HR'
line = 'day7'
col = stat_ref[stat][line]

def show_graph(df, stat):
    stat_ref = {'HR':{'day7':'HR_7DAYROLL', 'seas':'HR_CUMSUM'}
            , 'SB':{'day7': 'SB_7DAYROLL', 'seas':'SB_CUMSUM'}}
    # define graph vectors
    col_day7 = stat_ref[stat]['day7']
    col_seas = stat_ref[stat]['seas']
    seas     = df[col_seas]
    day7     = df[col_day7]
    x = df.DATE
    
    p = figure(title=stat+' summary'
           , x_axis_label='DATE'
           , y_axis_label=stat
           , x_axis_type='datetime'
           , y_axis_type='linear'
           , y_minor_ticks = 5
           , y_range=(0, 8)
           , plot_width = 900
           , plot_height = 600
          )
    
    p.line(x, day7, legend=stat+": 7-day", line_width=3, color="coral")
    #p.line(x, seas, legend=stat+": season", line_width=2, color="teal", alpha=.4)
    
    show(p)
    

In [ ]:
show_graph(test3, 'HR')

### Graph functionalities to add:

1. For stat and year, set the y limit to the max value for that column for that year (all players) -> dynamic upper y range (set lower y range to 0 in all cases

2. have a graph split in two windows, one for 7-day rolling, 1 for cumulative season (shared x-axis

3. Be able to specify player, year, and stat

4. Be able to save and export as a png and/or html